In [7]:
from aip import AipSpeech
from pyaudio import PyAudio,paInt16
import wave
import pygame
import requests
import json
import base64
import time

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [43]:
# 用网易云API搜索和下载歌曲
def search_song(wd):
    payload = {'key': '网易云API_key', 's': wd, 'limit': 5}
    r = requests.get('https://api.bzqll.com/music/netease/search', params=payload)
    return r.json()

def download_song(search_result):
    url = 'https://api.bzqll.com/music/netease/url?key=579621905&br=128000'+'&id='+search_result['data'][0]['id']
    r = requests.get(url) 
    with open("song\%s.mp3"%search_result['data'][0]['name'], "wb") as code:
        code.write(r.content)

In [2]:
# 用pygame播放音乐
def play_song(song_name):
    file='song\%s.mp3'%song_name
    pygame.mixer.init()
    track = pygame.mixer.music.load(file)
    pygame.mixer.music.play()
    if input('输入 stop 停止音乐：')=='stop':
        pygame.mixer.music.stop()

In [22]:
# 用pyaudio录音
def save_wave_file(filename,data):
    '''save the date to the wavfile'''
    channels=1
    sampwidth=2
    framerate=16000
    wf=wave.open(filename,'wb')
    wf.setnchannels(channels)#声道
    wf.setsampwidth(sampwidth)#采样字节 1 or 2
    wf.setframerate(framerate)#采样频率 8000 or 16000
    wf.writeframes(b"".join(data))#https://stackoverflow.com/questions/32071536/typeerror-sequence-item-0-expected-str-instance-bytes-found
    wf.close()

def my_record():
    framerate=16000
    NUM_SAMPLES=2000
    TIME=40
    pa=PyAudio()
    stream=pa.open(format = paInt16,channels=1,
                   rate=framerate,input=True,
                   frames_per_buffer=NUM_SAMPLES)
    my_buf=[]
    count=0
    while count<TIME:#控制录音时间
        string_audio_data = stream.read(NUM_SAMPLES)#一次性录音采样字节大小
        my_buf.append(string_audio_data)
        count+=1
        print('.')
    save_wave_file('myrecord.wav',my_buf)
    stream.close()

In [4]:
# 百度语音识别APK（十分简洁）
def speech_recognition(filePath):
    APP_ID = '百度key'
    API_KEY = '百度key'
    SECRET_KEY = '百度SECRET_KEY'

    client = AipSpeech(APP_ID, API_KEY, SECRET_KEY)
    
    with open(filePath, 'rb') as fp:
        return client.asr(fp.read(), 'wav', 16000, {'dev_pid': 1536,})

In [ ]:
# 讯飞语音识别（很复杂，故放弃）

#!/usr/bin/python
# -*- coding: UTF-8 -*-
#import urllib2
import time
import urllib
import json
import hashlib
import base64
import urllib.request
import urllib.parse
 

def speech_recognition():
    f = open("myrecord.wav", 'rb')  #rb表示二进制格式只读打开文件
    file_content = f.read()
    # file_content 是二进制内容，bytes类型
    # 由于Python的字符串类型是str，在内存中以Unicode表示，一个字符对应若干个字节。
    # 如果要在网络上传输，或者保存到磁盘上，就需要把str变为以字节为单位的bytes
    # 以Unicode表示的str通过encode()方法可以编码为指定的bytes
    base64_audio = base64.b64encode(file_content)   #base64.b64encode()参数是bytes类型，返回也是bytes类型
    body = urllib.parse.urlencode({'audio': base64_audio})
 
    url = 'http://api.xfyun.cn/v1/service/v1/iat'
    api_key = '讯飞key'
    param = {"engine_type": "sms16k", "aue": "raw"}
 
    x_appid = '讯飞id'
    x_param = base64.b64encode(json.dumps(param).replace(' ', '').encode('utf-8')) #改('''')
    # 这是3.x的用法，因为3.x中字符都为unicode编码，而b64encode函数的参数为byte类型，
    # 所以必须先转码为utf-8的bytes
    x_param = str(x_param, 'utf-8')
 
    x_time = int(int(round(time.time() * 1000)) / 1000)
    x_checksum = hashlib.md5((api_key + str(x_time) + x_param).encode('utf-8')).hexdigest() #改
    x_header = {'X-Appid': x_appid,
                'X-CurTime': x_time,
                'X-Param': x_param,
                'X-CheckSum': x_checksum}
    # 不要忘记url = ??, data = ??, headers = ??, method = ?? 中的“ = ”，这是python3
    req = urllib.request.Request(url = url, data = body.encode('utf-8'), headers = x_header, method = 'POST')
    result = urllib.request.urlopen(req)
    result = result.read().decode('utf-8')
    print(result)
    return
 
if __name__ == '__main__':
    speech_recognition()

In [5]:
# 讯飞歌曲识别（只能找到讯飞的）
def song_recognition(filePath):
    url = 'http://webqbh.xfyun.cn/v1/service/v1/qbh'
    api_key = '讯飞key'
    param = {"engine_type": "afs", "aue": "raw", "sample_rate": "16000"}
 
    x_appid = '讯飞id'
    x_param = base64.b64encode(json.dumps(param).replace(' ', '').encode('utf-8'))

    x_param = str(x_param, 'utf-8')
 
    x_time = int(int(round(time.time() * 1000)) / 1000)
    x_checksum = hashlib.md5((api_key + str(x_time) + x_param).encode('utf-8')).hexdigest()
    x_header = {'X-Appid': x_appid,
                'X-CurTime': str(x_time),
                'X-Param': x_param,
                'X-CheckSum': x_checksum}

    with open(filePath, 'rb') as file:
        r = requests.post(url = url, data = file.read(), headers = x_header)
    return json.loads(r.content.decode('utf-8'))

In [44]:
try:
    my_record()
    try:
        recognition_result = speech_recognition("myrecord.wav")['result'][0]
        try:
            search_result = search_song(recognition_result)
            singer = search_result['data'][0]['singer']
            song_name = search_result['data'][0]['name']
            try:
                with open('song\{}.mp3'.format(song_name)) as file:
                    file.close()
                print('正在播放  '+ singer + '-' +song_name)
                play_song(song_name)
            except:
                try:
                    download_song(search_result)
                    print('正在播放'  + singer + '-' +song_name)
                    play_song(song_name)
                except:
                    print('找不到歌曲')
        except:
            print('语音搜索失败')
    except:
        try:
            recognition_result = song_recognition("myrecord.wav")
            try:
                search_result = search_song(recognition_result['data'][0]['song'])
                singer = search_result['data'][0]['singer']
                song_name = search_result['data'][0]['name']
                try:
                    with open('song\{}.mp3'.format(song_name)) as file:
                        file.close()
                    print('正在播放  '+ singer + '-' +song_name)
                    play_song(song_name)
                except:
                    try:
                        download_song(search_result)
                        print('正在播放'  + singer + '-' +song_name)
                        play_song(song_name)
                    except:
                        print('找不到歌曲')
            except:
                print('曲调识曲失败')
        except:
            print('无法识别')
except:
    print('录音失败')

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
正在播放薛之谦-演员
输入 stop 停止音乐：stop
